In [2]:
import tensorflow as tf
print(tf.__version__)

1.14.0


Eager execution is enabled by default in tensorflow 2.x

if tensorflow version is 1.x then you need to execute the following line of code.

In [4]:
tf.enable_eager_execution()

if tensorflow_datasets is not installed, you could install it using the following cmd: pip install -q tensorflow-datasets

In [7]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

W0925 15:51:39.931115 140576904255232 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow_datasets/core/file_format_adapter.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /home/jupyter/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [8]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [13]:
training_sentenses = []
training_labels = []

testing_sentenses = []
testing_labels = []

# str(s.tonumpy()) is needed in python2 instead of just s.numpy()
for s,l in train_data:
    training_sentenses.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentenses.append(str(s.numpy()))
    testing_labels.append(l.numpy())        

In [27]:
# training_sentenses and training_labels are lists of Tensors: tf.Tensor()
print(training_sentenses[0])
print("label=",training_labels[0])
# convert list of tensors to numpy arrays
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

b"I saw this film on True Movies (which automatically made me sceptical) but actually - it was good. Why? Not because of the amazing plot twists or breathtaking dialogue (of which there is little) but because actually, despite what people say I thought the film was accurate in it's depiction of teenagers dealing with pregnancy.<br /><br />It's NOT Dawson's Creek, they're not graceful, cool witty characters who breeze through sexuality with effortless knowledge. They're kids and they act like kids would. <br /><br />They're blunt, awkward and annoyingly confused about everything. Yes, this could be by accident and they could just be bad actors but I don't think so. Dermot Mulroney gives (when not trying to be cool) a very believable performance and I loved him for it. Patricia Arquette IS whiny and annoying, but she was pregnant and a teenagers? The combination of the two isn't exactly lavender on your pillow. The plot was VERY predictable and but so what? I believed them, his stress an

In [20]:
%%time
vocab_size = 10000
embedding_dim=16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentenses)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentenses)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentenses)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, truncating=trunc_type)


CPU times: user 12.1 s, sys: 0 ns, total: 12.1 s
Wall time: 12.1 s


# define the Embedding model:

In [25]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, GlobalAveragePooling1D
model = tf.keras.Sequential([
    Embedding(vocab_size,embedding_dim,input_length=max_length),
    # instead of flatten, you can use GlobalAveragePooling1D() to reduce the
    # vector dimension and make your training faster
    Flatten(), 
    Dense(6,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [28]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

W0925 16:40:02.697649 140576904255232 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 12s 500us/sample - loss: 0.4910 - acc: 0.7473 - val_loss: 0.3808 - val_acc: 0.8319
Epoch 2/10
25000/25000 [==============================] - 11s 456us/sample - loss: 0.2452 - acc: 0.9050 - val_loss: 0.4178 - val_acc: 0.8160
Epoch 3/10
25000/25000 [==============================] - 11s 431us/sample - loss: 0.1008 - acc: 0.9745 - val_loss: 0.5193 - val_acc: 0.8027
Epoch 4/10
25000/25000 [==============================] - 10s 402us/sample - loss: 0.0285 - acc: 0.9961 - val_loss: 0.6264 - val_acc: 0.7979
Epoch 5/10
25000/25000 [==============================] - 11s 422us/sample - loss: 0.0110 - acc: 0.9988 - val_loss: 0.6679 - val_acc: 0.8023
Epoch 6/10
25000/25000 [==============================] - 11s 443us/sample - loss: 0.0046 - acc: 0.9995 - val_loss: 0.7269 - val_acc: 0.8043
Epoch 7/10
25000/25000 [==============================] - 11s 426us/sample - loss: 0.0018 - acc: 0.9998 

In [30]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentenses[1])

b'i have no idea what the other reviewer is talking about this was a wonderful movie and created a sense of the era that feels like time travel the characters are truly young mary is a strong match for byron claire is juvenile and a tad annoying <OOV> is a convincing beaten down <OOV> all are beautiful curious and <OOV> not the frightening <OOV> they are in gothic br br gothic works as an independent piece of shock film and i loved it for different reasons but this works like a <OOV> and <OOV> film and was from my <OOV> the best capture of what the summer must have felt like romantic yes but completely <OOV> my interest in the
b'I have no idea what the other reviewer is talking about- this was a wonderful movie, and created a sense of the era that feels like time travel. The characters are truly young, Mary is a strong match for Byron, Claire is juvenile and a tad annoying, Polidori is a convincing beaten-down sycophant... all are beautiful, curious, and decadent... not the frightening

# extract the embedding:

In [31]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [32]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()